In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/burnout-datathon-ieeecsmuj/sample_submission.csv
/kaggle/input/burnout-datathon-ieeecsmuj/val.csv
/kaggle/input/burnout-datathon-ieeecsmuj/train.csv
/kaggle/input/burnout-datathon-ieeecsmuj/test.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning Libraries
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
import xgboost as xgb
import lightgbm as lgb

# Advanced ML
from sklearn.ensemble import StackingRegressor, VotingRegressor
import optuna
from optuna.samplers import TPESampler
import gc



# 1. DATA LOADING AND EXPLORATION

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.feature_selection import SelectKBest, f_regression  
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import lightgbm as lgb

def main_pipeline():
    
    print("🏁 MOTOGP DATATHON - WINNING STRATEGY EXECUTION")
    print("="*60)
    
    print("📊 Loading Data...")
    train = pd.read_csv('/kaggle/input/burnout-datathon-ieeecsmuj/train.csv')
    test = pd.read_csv('/kaggle/input/burnout-datathon-ieeecsmuj/test.csv')
    
    print(f"Training data shape: {train.shape}")
    print(f"Test data shape: {test.shape}")
    print(f"Columns: {list(train.columns)}")

    print(f"\n🎯 Target Analysis:")
    print(f"Mean lap time: {train['Lap_Time_Seconds'].mean():.2f}s")
    print(f"Std lap time: {train['Lap_Time_Seconds'].std():.2f}s")
    
    return train, test


train, test = main_pipeline()

🏁 MOTOGP DATATHON - WINNING STRATEGY EXECUTION
📊 Loading Data...
Training data shape: (1914056, 45)
Test data shape: (546874, 44)
Columns: ['Unique ID', 'Rider_ID', 'category_x', 'Circuit_Length_km', 'Laps', 'Grid_Position', 'Avg_Speed_kmh', 'Track_Condition', 'Humidity_%', 'Tire_Compound_Front', 'Tire_Compound_Rear', 'Penalty', 'Championship_Points', 'Championship_Position', 'Session', 'year_x', 'sequence', 'rider', 'team', 'bike', 'position', 'points', 'shortname', 'circuit_name', 'rider_name', 'team_name', 'bike_name', 'Lap_Time_Seconds', 'Corners_per_Lap', 'Tire_Degradation_Factor_per_Lap', 'Pit_Stop_Duration_Seconds', 'Ambient_Temperature_Celsius', 'Track_Temperature_Celsius', 'weather', 'track', 'air', 'ground', 'starts', 'finishes', 'with_points', 'podiums', 'wins', 'min_year', 'max_year', 'years_active']

🎯 Target Analysis:
Mean lap time: 90.00s
Std lap time: 11.53s


In [4]:
def load_and_explore_data():
    print("\n📊 Loading Data...")
    
    train = pd.read_csv('/kaggle/input/burnout-datathon-ieeecsmuj/train.csv')
    test = pd.read_csv('/kaggle/input/burnout-datathon-ieeecsmuj/test.csv')
    
    print(f"Training data shape: {train.shape}")
    print(f"Test data shape: {test.shape}")
    print(f"Target variable: Lap_Time_Seconds")
    
    print(f"\nTarget Statistics:")
    print(f"Mean: {train['Lap_Time_Seconds'].mean():.2f} seconds")
    print(f"Std: {train['Lap_Time_Seconds'].std():.2f} seconds")
    print(f"Min: {train['Lap_Time_Seconds'].min():.2f} seconds")
    print(f"Max: {train['Lap_Time_Seconds'].max():.2f} seconds")
    
    print(f"\nMissing values in train: {train.isnull().sum().sum()}")
    print(f"Missing values in test: {test.isnull().sum().sum()}")
    
    return train, test
train, test = load_and_explore_data()


📊 Loading Data...
Training data shape: (1914056, 45)
Test data shape: (546874, 44)
Target variable: Lap_Time_Seconds

Target Statistics:
Mean: 90.00 seconds
Std: 11.53 seconds
Min: 70.00 seconds
Max: 110.00 seconds

Missing values in train: 321292
Missing values in test: 91555


# PHASE 2: FEATURE ENGINEERING

In [5]:
print("Available columns in train dataset:")
print(train.columns.tolist())
print(f"\nDataFrame shape: {train.shape}")

print("\nFirst few rows:")
print(train.head())

print("\nColumns containing 'start' (case insensitive):")
start_cols = [col for col in train.columns if 'start' in col.lower()]
print(start_cols)

print("\nColumns containing 'race' (case insensitive):")
race_cols = [col for col in train.columns if 'race' in col.lower()]  
print(race_cols)

print("\nColumns containing 'finish' (case insensitive):")
finish_cols = [col for col in train.columns if 'finish' in col.lower()]
print(finish_cols)
def phase2_feature_engineering_corrected(train_df, test_df):
    
    print("\n🛠️ PHASE 2: FEATURE ENGINEERING (CORRECTED)")
    print("="*50)
    
    def create_features(df):
        df = df.copy()
        available_cols = df.columns.tolist()
        print(f"Working with {len(available_cols)} columns")
        
        if 'Championship_Points' in available_cols and 'starts' in available_cols:
            df['points_per_start'] = df['Championship_Points'] / (df['starts'] + 1)
        
        if 'podiums' in available_cols and 'finishes' in available_cols:
            df['podium_rate'] = df['podiums'] / (df['finishes'] + 1)
            
        if 'wins' in available_cols and 'finishes' in available_cols:
            df['win_rate'] = df['wins'] / (df['finishes'] + 1)
            
        if 'finishes' in available_cols and 'starts' in available_cols:
            df['finish_rate'] = df['finishes'] / (df['starts'] + 1)
        
        if all(col in available_cols for col in ['Corners_per_Lap', 'Circuit_Length_km']):
            df['corners_per_km'] = df['Corners_per_Lap'] / (df['Circuit_Length_km'] + 0.001)
            df['distance_per_corner'] = df['Circuit_Length_km'] / (df['Corners_per_Lap'] + 1)
            
        if all(col in available_cols for col in ['Circuit_Length_km', 'Laps']):
            df['total_race_distance'] = df['Circuit_Length_km'] * df['Laps']
        
        if all(col in available_cols for col in ['Track_Temperature_Celsius', 'Ambient_Temperature_Celsius']):
            df['temp_diff'] = df['Track_Temperature_Celsius'] - df['Ambient_Temperature_Celsius']
            
        if all(col in available_cols for col in ['Avg_Speed_kmh', 'Track_Temperature_Celsius']):
            df['speed_temp_interaction'] = df['Avg_Speed_kmh'] * df['Track_Temperature_Celsius']
            
        if all(col in available_cols for col in ['Humidity_%', 'Ambient_Temperature_Celsius']):
            df['humidity_temp_interaction'] = df['Humidity_%'] * df['Ambient_Temperature_Celsius']
        
        if 'Grid_Position' in available_cols:
            df['grid_advantage'] = 1 / (df['Grid_Position'] + 1)
            
        if all(col in available_cols for col in ['Championship_Points', 'Championship_Position']):
            df['championship_momentum'] = df['Championship_Points'] / (df['Championship_Position'] + 1)
            df['position_pressure'] = df['Championship_Position'] * df['Grid_Position'] if 'Grid_Position' in available_cols else df['Championship_Position']
        
        if all(col in available_cols for col in ['Tire_Compound_Front', 'Tire_Compound_Rear']):
            df['tire_strategy'] = df['Tire_Compound_Front'].astype(str) + '_' + df['Tire_Compound_Rear'].astype(str)
            
        if all(col in available_cols for col in ['Tire_Degradation_Factor_per_Lap', 'Laps']):
            df['tire_degradation_total'] = df['Tire_Degradation_Factor_per_Lap'] * df['Laps']
        
        if all(col in available_cols for col in ['years_active', 'Championship_Points']):
            df['experience_points_ratio'] = df['years_active'] * df['Championship_Points']
            
        if all(col in available_cols for col in ['starts', 'years_active']):
            df['starts_per_year'] = df['starts'] / (df['years_active'] + 1)
        
        if all(col in available_cols for col in ['Avg_Speed_kmh', 'Corners_per_Lap']):
            df['speed_per_corner'] = df['Avg_Speed_kmh'] / (df['Corners_per_Lap'] + 1)
            
        if all(col in available_cols for col in ['Circuit_Length_km', 'Avg_Speed_kmh']):
            df['theoretical_lap_time'] = (df['Circuit_Length_km'] * 3600) / (df['Avg_Speed_kmh'] + 1)
        
        return df
    
    train_engineered = create_features(train_df)
    test_engineered = create_features(test_df)
    
    print(f"\n✅ Features created! Train shape: {train_engineered.shape}")
    
    return train_engineered, test_engineered

train_engineered, test_engineered = phase2_feature_engineering_corrected(train, test)
print(f"\n🎯 New feature count: {train_engineered.shape[1] - train.shape[1]} features added")
print(f"Total features now: {train_engineered.shape[1]}")

original_cols = set(train.columns)
new_cols = set(train_engineered.columns) - original_cols
print(f"\n🆕 New features created: {len(new_cols)}")
for col in sorted(new_cols):
    print(f"   - {col}")

Available columns in train dataset:
['Unique ID', 'Rider_ID', 'category_x', 'Circuit_Length_km', 'Laps', 'Grid_Position', 'Avg_Speed_kmh', 'Track_Condition', 'Humidity_%', 'Tire_Compound_Front', 'Tire_Compound_Rear', 'Penalty', 'Championship_Points', 'Championship_Position', 'Session', 'year_x', 'sequence', 'rider', 'team', 'bike', 'position', 'points', 'shortname', 'circuit_name', 'rider_name', 'team_name', 'bike_name', 'Lap_Time_Seconds', 'Corners_per_Lap', 'Tire_Degradation_Factor_per_Lap', 'Pit_Stop_Duration_Seconds', 'Ambient_Temperature_Celsius', 'Track_Temperature_Celsius', 'weather', 'track', 'air', 'ground', 'starts', 'finishes', 'with_points', 'podiums', 'wins', 'min_year', 'max_year', 'years_active']

DataFrame shape: (1914056, 45)

First few rows:
   Unique ID  Rider_ID category_x  Circuit_Length_km  Laps  Grid_Position  \
0    1894944      2659      Moto2              4.874    22             17   
1      23438      5205      Moto2              3.875    24              7   

# PHASE 3:Advanced Preprocessing

In [6]:
print("\n" + "🚀" * 25 + " STARTING PIPELINE EXECUTION " + "🚀" * 25)

print("\n🔧 PHASE 3: ADVANCED PREPROCESSING")
print("="*50)

train_processed = train_engineered.copy()
test_processed = test_engineered.copy()

if 'Lap_Time_Seconds' in train_processed.columns:
    y = train_processed['Lap_Time_Seconds']
    train_processed = train_processed.drop('Lap_Time_Seconds', axis=1)
    print("✅ Target variable separated")

test_ids = test_processed['Unique_ID'] if 'Unique_ID' in test_processed.columns else None
print(f"✅ Test IDs stored: {test_ids is not None}")

id_cols = ['Unique_ID', 'id', 'ID']
for col in id_cols:
    if col in train_processed.columns:
        train_processed = train_processed.drop(col, axis=1)
    if col in test_processed.columns:
        test_processed = test_processed.drop(col, axis=1)


categorical_cols = train_processed.select_dtypes(include=['object']).columns.tolist()
print(f"✅ Categorical columns found: {len(categorical_cols)}: {categorical_cols}")

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    # Combine train and test for consistent encoding
    combined_values = pd.concat([train_processed[col], test_processed[col]]).astype(str)
    le.fit(combined_values)
    
    train_processed[col] = le.transform(train_processed[col].astype(str))
    test_processed[col] = le.transform(test_processed[col].astype(str))
    label_encoders[col] = le
    print(f"   ✓ Encoded {col}: {len(le.classes_)} unique values")

numeric_cols = train_processed.select_dtypes(include=[np.number]).columns
train_processed[numeric_cols] = train_processed[numeric_cols].fillna(train_processed[numeric_cols].median())
test_processed[numeric_cols] = test_processed[numeric_cols].fillna(train_processed[numeric_cols].median())

train_processed = train_processed.replace([np.inf, -np.inf], np.nan)
test_processed = test_processed.replace([np.inf, -np.inf], np.nan)
train_processed = train_processed.fillna(train_processed.median())
test_processed = test_processed.fillna(train_processed.median())

print(f"✅ Preprocessing complete!")
print(f"Final train shape: {train_processed.shape}")
print(f"Final test shape: {test_processed.shape}")



🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀 STARTING PIPELINE EXECUTION 🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀

🔧 PHASE 3: ADVANCED PREPROCESSING
✅ Target variable separated
✅ Test IDs stored: False
✅ Categorical columns found: 14: ['category_x', 'Track_Condition', 'Tire_Compound_Front', 'Tire_Compound_Rear', 'Penalty', 'Session', 'shortname', 'circuit_name', 'rider_name', 'team_name', 'bike_name', 'weather', 'track', 'tire_strategy']
   ✓ Encoded category_x: 3 unique values
   ✓ Encoded Track_Condition: 2 unique values
   ✓ Encoded Tire_Compound_Front: 3 unique values
   ✓ Encoded Tire_Compound_Rear: 3 unique values
   ✓ Encoded Penalty: 6 unique values
   ✓ Encoded Session: 7 unique values
   ✓ Encoded shortname: 53 unique values
   ✓ Encoded circuit_name: 70 unique values
   ✓ Encoded rider_name: 2695 unique values
   ✓ Encoded team_name: 967 unique values
   ✓ Encoded bike_name: 301 unique values
   ✓ Encoded weather: 5 unique values
   ✓ Encoded track: 2 unique values
   ✓ Encoded tire_strategy: 9 unique value

# Phase 4: Feature Selection

In [7]:
print(f"\n🎯 PHASE 4: FEATURE SELECTION (Top 50)")
print("="*50)

selector = SelectKBest(score_func=f_regression, k=min(50, train_processed.shape[1]))
X_train_selected = selector.fit_transform(train_processed, y)
X_test_selected = selector.transform(test_processed)

selected_features = train_processed.columns[selector.get_support()].tolist()
feature_scores = selector.scores_[selector.get_support()]

print(f"✅ Selected {len(selected_features)} features out of {train_processed.shape[1]}")
print("\n🏆 Top 10 Most Important Features:")

feature_importance = pd.DataFrame({
    'feature': selected_features,
    'score': feature_scores
}).sort_values('score', ascending=False)

for i, row in feature_importance.head(10).iterrows():
    print(f"   {i+1:2d}. {row['feature']}: {row['score']:.2f}")

X_train_selected = pd.DataFrame(X_train_selected, columns=selected_features)
X_test_selected = pd.DataFrame(X_test_selected, columns=selected_features)

print(f"\n✅ Phase 3 & 4 completed successfully!")
print(f"📊 Ready for model training with {len(selected_features)} selected features")


🎯 PHASE 4: FEATURE SELECTION (Top 50)
✅ Selected 50 features out of 63

🏆 Top 10 Most Important Features:
   19. team_name: 151.22
   21. Corners_per_Lap: 138.75
   32. with_points: 127.37
   42. corners_per_km: 116.22
   12. team: 104.04
   37. years_active: 100.23
   15. points: 96.11
   43. distance_per_corner: 85.05
   41. finish_rate: 84.81
   31. finishes: 72.04

✅ Phase 3 & 4 completed successfully!
📊 Ready for model training with 50 selected features


# Phase 5: Advanced Model Development


In [8]:
print("\n🤖 PHASE 5: ADVANCED MODEL DEVELOPMENT")
print("="*50)

# Base models with optimized parameters
models = {
    'Random Forest': RandomForestRegressor(
        n_estimators=100,  # Reduced for large dataset
        max_depth=15,
        min_samples_split=10,  # Increased for large dataset
        min_samples_leaf=5,    # Increased for large dataset
        random_state=42,
        n_jobs=-1
    ),
    
    'XGBoost': xgb.XGBRegressor(
        n_estimators=100,   # Reduced for large dataset
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    ),
    
    'LightGBM': lgb.LGBMRegressor(
        n_estimators=100,   # Reduced for large dataset
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    ),
    
    'Ridge': Ridge(alpha=10.0),
    
    'Lasso': Lasso(alpha=1.0, max_iter=2000)
}

print(f"✅ Created {len(models)} models:")
for name in models.keys():
    print(f"   ✓ {name}")


🤖 PHASE 5: ADVANCED MODEL DEVELOPMENT
✅ Created 5 models:
   ✓ Random Forest
   ✓ XGBoost
   ✓ LightGBM
   ✓ Ridge
   ✓ Lasso


# Phase 6: Model Training & Evaluation

In [9]:
print("\n🏋️ PHASE 6: MODEL TRAINING SETUP")
print("="*50)

import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

results = {}
trained_models = {}

max_train_size = 100000
if len(X_train_selected) > max_train_size:
    sample_idx = np.random.choice(len(X_train_selected), max_train_size, replace=False)
    X_train_fast = X_train_selected.iloc[sample_idx]
    y_train_fast = y.iloc[sample_idx]
    print(f"✅ Reduced training set to {max_train_size:,} samples")
else:
    X_train_fast = X_train_selected
    y_train_fast = y

# Single train-validation split
X_train_sub, X_val_sub, y_train_sub, y_val_sub = train_test_split(
    X_train_fast, y_train_fast, test_size=0.2, random_state=42
)

print(f"📊 Training samples: {len(X_train_sub):,}")
print(f"📊 Validation samples: {len(X_val_sub):,}")
print(f"📊 Features: {X_train_sub.shape[1]}")


🏋️ PHASE 6: MODEL TRAINING SETUP
✅ Reduced training set to 100,000 samples
📊 Training samples: 80,000
📊 Validation samples: 20,000
📊 Features: 50


In [10]:
print("\n🚀 DEFINING FAST MODELS")
print("-" * 30)

fast_models = {
    'Ridge': Ridge(alpha=1.0),
    
    'RandomForest_Fast': RandomForestRegressor(
        n_estimators=50,
        max_depth=10,
        min_samples_split=20,
        min_samples_leaf=10,
        n_jobs=-1,
        random_state=42
    ),
    
    'XGBoost_Fast': xgb.XGBRegressor(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        n_jobs=-1,
        random_state=42
    ),
    
    'LightGBM_Fast': lgb.LGBMRegressor(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        n_jobs=-1,
        random_state=42,
        verbose=-1
    )
}

print(f"✅ Defined {len(fast_models)} fast models")


🚀 DEFINING FAST MODELS
------------------------------
✅ Defined 4 fast models


In [11]:
# MODEL TRAINING LOOP
print("\n🔄 TRAINING MODELS")
print("-" * 30)

for name, model in fast_models.items():
    print(f"\n🔄 Training {name}...")
    
    try:
        start_time = time.time()
        model.fit(X_train_sub, y_train_sub)
        
        val_pred = model.predict(X_val_sub)
        val_rmse = np.sqrt(mean_squared_error(y_val_sub, val_pred))
        
        training_time = time.time() - start_time
        
        results[name] = {
            'Val_RMSE': val_rmse,
            'Training_Time': training_time,
            'model': model
        }
        trained_models[name] = model
        
        print(f"   ✅ RMSE: {val_rmse:.4f} | Time: {training_time:.1f}s")
        
        gc.collect()
        
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
        continue

print(f"\n📊 TRAINING SUMMARY")
print("-" * 40)
for name, info in results.items():
    print(f"{name:<20}: RMSE={info['Val_RMSE']:.4f}, Time={info['Training_Time']:.1f}s")


🔄 TRAINING MODELS
------------------------------

🔄 Training Ridge...
   ✅ RMSE: 11.4611 | Time: 0.1s

🔄 Training RandomForest_Fast...
   ✅ RMSE: 11.2374 | Time: 20.8s

🔄 Training XGBoost_Fast...
   ✅ RMSE: 11.1059 | Time: 1.0s

🔄 Training LightGBM_Fast...
   ✅ RMSE: 11.2109 | Time: 0.8s

📊 TRAINING SUMMARY
----------------------------------------
Ridge               : RMSE=11.4611, Time=0.1s
RandomForest_Fast   : RMSE=11.2374, Time=20.8s
XGBoost_Fast        : RMSE=11.1059, Time=1.0s
LightGBM_Fast       : RMSE=11.2109, Time=0.8s


In [12]:
# BEST MODEL SELECTION
print("\n🏆 SELECTING BEST MODEL")
print("-" * 30)

if results:
    best_model_name = min(results.keys(), key=lambda x: results[x]['Val_RMSE'])
    best_model = trained_models[best_model_name]
    best_rmse = results[best_model_name]['Val_RMSE']
    
    print(f"🏆 BEST MODEL: {best_model_name}")
    print(f"🎯 Best RMSE: {best_rmse:.4f}")
    
    print(f"\n📈 Final training on {len(X_train_fast):,} samples...")
    start_time = time.time()
    best_model.fit(X_train_fast, y_train_fast)
    final_training_time = time.time() - start_time
    print(f"✅ Final training completed in {final_training_time:.1f}s")
    
else:
    print("❌ No models trained successfully, using Ridge fallback")
    best_model_name = "Ridge"
    best_model = Ridge(alpha=1.0)
    best_model.fit(X_train_fast, y_train_fast)

print(f"\n🎯 READY FOR PREDICTIONS WITH: {best_model_name}")


🏆 SELECTING BEST MODEL
------------------------------
🏆 BEST MODEL: XGBoost_Fast
🎯 Best RMSE: 11.1059

📈 Final training on 100,000 samples...
✅ Final training completed in 1.1s

🎯 READY FOR PREDICTIONS WITH: XGBoost_Fast


# PHASE 7: GENERATE PREDICTIONS

In [13]:
print("\n🎯 PHASE 7: GENERATE PREDICTIONS")
print("="*50)

print("🔮 Generating predictions...")
predictions = best_model.predict(X_test_selected)

print(f"\n📊 PREDICTION STATISTICS:")
print(f"   Mean: {predictions.mean():.2f} seconds")
print(f"   Std:  {predictions.std():.2f} seconds")
print(f"   Min:  {predictions.min():.2f} seconds")
print(f"   Max:  {predictions.max():.2f} seconds")

# Handle test IDs
if 'test_ids' in locals() and test_ids is not None:
    submission_ids = test_ids
else:
    submission_ids = range(len(predictions))

submission = pd.DataFrame({
    'Unique_ID': submission_ids,
    'Lap_Time_Seconds': predictions
})

submission.to_csv('solution.csv', index=False)

print(f"\n✅ BASIC SUBMISSION CREATED")
print(f"📁 File: solution.csv")
print(f"📊 Shape: {submission.shape}")
print(f"🏆 Model: {best_model_name}")

print(f"\n📋 FIRST 5 PREDICTIONS:")
print(submission.head())


🎯 PHASE 7: GENERATE PREDICTIONS
🔮 Generating predictions...

📊 PREDICTION STATISTICS:
   Mean: 90.04 seconds
   Std:  1.49 seconds
   Min:  77.22 seconds
   Max:  102.06 seconds

✅ BASIC SUBMISSION CREATED
📁 File: solution.csv
📊 Shape: (546874, 2)
🏆 Model: XGBoost_Fast

📋 FIRST 5 PREDICTIONS:
   Unique_ID  Lap_Time_Seconds
0          0         90.442490
1          1         91.103645
2          2         89.846092
3          3         90.351425
4          4         91.091522


# PHASE 8: FAST ENSEMBLE 

In [14]:

print("\n🚀 PHASE 8: FAST ENSEMBLE")
print("="*40)

ensemble_preds = []
ensemble_weights = []

print("🔄 Generating ensemble predictions...")

for name, model_info in results.items():
    if name != best_model_name:
        try:
            pred = trained_models[name].predict(X_test_selected)
            ensemble_preds.append(pred)
            weight = 1.0 / model_info['Val_RMSE']
            ensemble_weights.append(weight)
            print(f"   ✅ {name}: weight={weight:.3f}")
        except Exception as e:
            print(f"   ❌ {name}: {e}")
            continue

if len(ensemble_preds) >= 2:
    ensemble_weights = np.array(ensemble_weights)
    ensemble_weights = ensemble_weights / ensemble_weights.sum()
    
    blended_pred = np.zeros(len(predictions))
    for i, pred in enumerate(ensemble_preds):
        blended_pred += ensemble_weights[i] * pred
    
    best_weight = 0.5
    final_pred = best_weight * predictions + (1 - best_weight) * blended_pred
    
    print(f"\n✅ ENSEMBLE CREATED")
    print(f"📊 Models combined: {len(ensemble_preds)+1}")
    print(f"🏆 Best model weight: {best_weight}")
    
    ensemble_submission = pd.DataFrame({
        'Unique_ID': submission_ids,
        'Lap_Time_Seconds': final_pred
    })
    
    ensemble_submission.to_csv('solution_ensemble.csv', index=False)
    
    print(f"\n📊 ENSEMBLE STATISTICS:")
    print(f"   Mean: {final_pred.mean():.2f} seconds")
    print(f"   Std:  {final_pred.std():.2f} seconds")
    print(f"   Min:  {final_pred.min():.2f} seconds")
    print(f"   Max:  {final_pred.max():.2f} seconds")
    
    print(f"\n✅ ENSEMBLE SUBMISSION CREATED")
    print(f"📁 File: solution_ensemble.csv")
    
else:
    print("⚠️ Not enough models for ensemble, using single model")

print(f"\n🏁 PIPELINE COMPLETED")
print(f"📁 Files created: solution.csv" + (", solution_ensemble.csv" if len(ensemble_preds) >= 2 else ""))
print("🚀 Ready for Kaggle submission!")


🚀 PHASE 8: FAST ENSEMBLE
🔄 Generating ensemble predictions...
   ✅ Ridge: weight=0.087
   ✅ RandomForest_Fast: weight=0.089
   ✅ LightGBM_Fast: weight=0.089

✅ ENSEMBLE CREATED
📊 Models combined: 4
🏆 Best model weight: 0.5

📊 ENSEMBLE STATISTICS:
   Mean: 90.05 seconds
   Std:  0.98 seconds
   Min:  81.24 seconds
   Max:  98.52 seconds

✅ ENSEMBLE SUBMISSION CREATED
📁 File: solution_ensemble.csv

🏁 PIPELINE COMPLETED
📁 Files created: solution.csv, solution_ensemble.csv
🚀 Ready for Kaggle submission!


# ADVANCED METHODS:

# PHASE 9: ADVANCED FEATURE ENGINEERING

In [15]:
print("\n🧠 PHASE 9: ADVANCED FEATURE ENGINEERING")
print("="*50)

def create_winning_features(X_train, X_test, y_train):
        
    X_train_adv = X_train.copy()
    X_test_adv = X_test.copy()
    
    print("🔧 Creating polynomial features...")
    from sklearn.preprocessing import PolynomialFeatures
    
    numeric_cols = X_train.select_dtypes(include=[np.number]).columns[:10]
    poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
    
    X_train_poly = poly.fit_transform(X_train[numeric_cols])
    X_test_poly = poly.transform(X_test[numeric_cols])
    
    poly_feature_names = [f"poly_{i}" for i in range(X_train_poly.shape[1])]
    
    for i, name in enumerate(poly_feature_names):
        X_train_adv[name] = X_train_poly[:, i]
        X_test_adv[name] = X_test_poly[:, i]
    
    print(f"✅ Added {len(poly_feature_names)} polynomial features")
    
    print("🎯 Creating target encoding features...")
    categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns
    
    for col in categorical_cols[:5]:  # Limit to top 5 categorical
        if col in X_train.columns:
            target_mean = X_train.groupby(col)[y_train.name if hasattr(y_train, 'name') else 'target'].transform('mean')
            X_train_adv[f'{col}_target_enc'] = target_mean
            
            # For test, use global mean for unseen categories
            col_means = pd.Series(y_train).groupby(X_train[col]).mean()
            X_test_adv[f'{col}_target_enc'] = X_test[col].map(col_means).fillna(y_train.mean())
    
    print(f"✅ Added target encoding for {min(len(categorical_cols), 5)} categorical features")
    
    print("📊 Creating statistical features...")
    numeric_features = X_train_adv.select_dtypes(include=[np.number]).columns
    
    if len(numeric_features) > 5:
        feature_subset = numeric_features[:15]  # Top 15 numeric features
        
        # Row-wise statistics
        X_train_adv['row_mean'] = X_train_adv[feature_subset].mean(axis=1)
        X_train_adv['row_std'] = X_train_adv[feature_subset].std(axis=1)
        X_train_adv['row_max'] = X_train_adv[feature_subset].max(axis=1)
        X_train_adv['row_min'] = X_train_adv[feature_subset].min(axis=1)
        X_train_adv['row_range'] = X_train_adv['row_max'] - X_train_adv['row_min']
        
        X_test_adv['row_mean'] = X_test_adv[feature_subset].mean(axis=1)
        X_test_adv['row_std'] = X_test_adv[feature_subset].std(axis=1)
        X_test_adv['row_max'] = X_test_adv[feature_subset].max(axis=1)
        X_test_adv['row_min'] = X_test_adv[feature_subset].min(axis=1)
        X_test_adv['row_range'] = X_test_adv['row_max'] - X_test_adv['row_min']
        
        print("✅ Added 5 statistical features")
    
    print("🏁 Creating domain-specific racing features...")
    if 'Grid_Position' in X_train_adv.columns and 'Championship_Position' in X_train_adv.columns:
        X_train_adv['pressure_index'] = X_train_adv['Grid_Position'] * X_train_adv['Championship_Position']
        X_test_adv['pressure_index'] = X_test_adv['Grid_Position'] * X_test_adv['Championship_Position']
    
    if 'Avg_Speed_kmh' in X_train_adv.columns and 'Circuit_Length_km' in X_train_adv.columns:
        X_train_adv['speed_per_km'] = X_train_adv['Avg_Speed_kmh'] / (X_train_adv['Circuit_Length_km'] + 0.1)
        X_test_adv['speed_per_km'] = X_test_adv['Avg_Speed_kmh'] / (X_test_adv['Circuit_Length_km'] + 0.1)
    
    if 'Track_Temperature_Celsius' in X_train_adv.columns and 'Humidity_%' in X_train_adv.columns:
        X_train_adv['weather_factor'] = X_train_adv['Track_Temperature_Celsius'] * X_train_adv['Humidity_%'] / 100
        X_test_adv['weather_factor'] = X_test_adv['Track_Temperature_Celsius'] * X_test_adv['Humidity_%'] / 100
    
    print("✅ Added domain-specific features")
    
    print("🎯 Selecting best features...")
    from sklearn.feature_selection import SelectKBest, f_regression
    
    X_train_adv = X_train_adv.replace([np.inf, -np.inf], np.nan)
    X_test_adv = X_test_adv.replace([np.inf, -np.inf], np.nan)
    
    X_train_adv = X_train_adv.fillna(X_train_adv.median())
    X_test_adv = X_test_adv.fillna(X_train_adv.median())
    
    max_features = min(500, X_train_adv.shape[1])  # Max 500 features
    selector = SelectKBest(score_func=f_regression, k=max_features)
    
    X_train_selected = selector.fit_transform(X_train_adv, y_train)
    X_test_selected = selector.transform(X_test_adv)
    
    selected_feature_names = X_train_adv.columns[selector.get_support()].tolist()
    
    print(f"✅ Selected {len(selected_feature_names)} best features")
    print(f"📊 Final feature count: {X_train_selected.shape[1]}")
    
    return X_train_selected, X_test_selected, selected_feature_names

print("🚀 Applying advanced feature engineering...")
X_train_advanced, X_test_advanced, advanced_features = create_winning_features(
    pd.DataFrame(X_train_fast, columns=selected_features), 
    pd.DataFrame(X_test_selected, columns=selected_features),
    y_train_fast
)

print(f"\n🎯 ADVANCED FEATURES READY")
print(f"📊 Training shape: {X_train_advanced.shape}")
print(f"📊 Test shape: {X_test_advanced.shape}")
print(f"🚀 Feature count increased from {len(selected_features)} to {len(advanced_features)}")


🧠 PHASE 9: ADVANCED FEATURE ENGINEERING
🚀 Applying advanced feature engineering...
🔧 Creating polynomial features...
✅ Added 55 polynomial features
🎯 Creating target encoding features...
✅ Added target encoding for 0 categorical features
📊 Creating statistical features...
✅ Added 5 statistical features
🏁 Creating domain-specific racing features...
✅ Added domain-specific features
🎯 Selecting best features...
✅ Selected 111 best features
📊 Final feature count: 111

🎯 ADVANCED FEATURES READY
📊 Training shape: (100000, 111)
📊 Test shape: (546874, 111)
🚀 Feature count increased from 50 to 111


# PHASE 10: COMPETITION-WINNING MODELS

In [16]:

print("\n🏆 PHASE 10: COMPETITION-WINNING MODELS")
print("="*50)

from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

# Advanced hyperparameter-tuned models
winning_models = {
    'XGBoost_Tuned': xgb.XGBRegressor(
        n_estimators=300,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.85,
        colsample_bytree=0.85,
        reg_alpha=0.1,
        reg_lambda=0.1,
        min_child_weight=3,
        gamma=0.1,
        random_state=42,
        n_jobs=-1
    ),
    
    'LightGBM_Tuned': lgb.LGBMRegressor(
        n_estimators=300,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.85,
        colsample_bytree=0.85,
        reg_alpha=0.1,
        reg_lambda=0.1,
        min_child_samples=20,
        num_leaves=31,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    ),
    
    'ExtraTrees_Tuned': ExtraTreesRegressor(
        n_estimators=200,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=3,
        max_features='sqrt',
        bootstrap=False,
        random_state=42,
        n_jobs=-1
    ),
    
    'GradientBoosting_Tuned': GradientBoostingRegressor(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.85,
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42
    ),
    
    'RandomForest_Tuned': RandomForestRegressor(
        n_estimators=200,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=3,
        max_features='sqrt',
        bootstrap=True,
        random_state=42,
        n_jobs=-1
    )
}

print(f"🚀 Defined {len(winning_models)} competition-grade models")

# Train advanced models
advanced_results = {}
advanced_trained_models = {}

# Create train-validation split for advanced features
X_train_adv_sub, X_val_adv_sub, y_train_adv_sub, y_val_adv_sub = train_test_split(
    X_train_advanced, y_train_fast, test_size=0.2, random_state=42
)

print(f"\n🔄 TRAINING ADVANCED MODELS")
print("-" * 40)

for name, model in winning_models.items():
    print(f"\n🔄 Training {name}...")
    
    try:
        start_time = time.time()
        model.fit(X_train_adv_sub, y_train_adv_sub)
        
        val_pred = model.predict(X_val_adv_sub)
        val_rmse = np.sqrt(mean_squared_error(y_val_adv_sub, val_pred))
        
        training_time = time.time() - start_time
        
        advanced_results[name] = {
            'Val_RMSE': val_rmse,
            'Training_Time': training_time,
            'model': model
        }
        advanced_trained_models[name] = model
        
        print(f"   ✅ RMSE: {val_rmse:.4f} | Time: {training_time:.1f}s")
        
        gc.collect()
        
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
        continue

print(f"\n🏆 ADVANCED MODEL RESULTS")
print("-" * 50)
sorted_models = sorted(advanced_results.items(), key=lambda x: x[1]['Val_RMSE'])
for name, info in sorted_models:
    print(f"{name:<25}: RMSE={info['Val_RMSE']:.4f}, Time={info['Training_Time']:.1f}s")

# Select best advanced model
if advanced_results:
    best_advanced_name = min(advanced_results.keys(), key=lambda x: advanced_results[x]['Val_RMSE'])
    best_advanced_model = advanced_trained_models[best_advanced_name]
    best_advanced_rmse = advanced_results[best_advanced_name]['Val_RMSE']
    
    print(f"\n🏆 BEST ADVANCED MODEL: {best_advanced_name}")
    print(f"🎯 Best RMSE: {best_advanced_rmse:.4f}")
    
    # Final training on all advanced data
    print(f"📈 Final training on advanced features...")
    best_advanced_model.fit(X_train_advanced, y_train_fast)
    
else:
    best_advanced_name = "XGBoost_Tuned"
    best_advanced_model = winning_models[best_advanced_name]
    best_advanced_model.fit(X_train_advanced, y_train_fast)
    print("✅ Using XGBoost fallback")


🏆 PHASE 10: COMPETITION-WINNING MODELS
🚀 Defined 5 competition-grade models

🔄 TRAINING ADVANCED MODELS
----------------------------------------

🔄 Training XGBoost_Tuned...
   ✅ RMSE: 10.8094 | Time: 16.6s

🔄 Training LightGBM_Tuned...
   ✅ RMSE: 11.1779 | Time: 3.1s

🔄 Training ExtraTrees_Tuned...
   ✅ RMSE: 11.3727 | Time: 9.6s

🔄 Training GradientBoosting_Tuned...
   ✅ RMSE: 11.2624 | Time: 631.5s

🔄 Training RandomForest_Tuned...
   ✅ RMSE: 11.3508 | Time: 31.7s

🏆 ADVANCED MODEL RESULTS
--------------------------------------------------
XGBoost_Tuned            : RMSE=10.8094, Time=16.6s
LightGBM_Tuned           : RMSE=11.1779, Time=3.1s
GradientBoosting_Tuned   : RMSE=11.2624, Time=631.5s
RandomForest_Tuned       : RMSE=11.3508, Time=31.7s
ExtraTrees_Tuned         : RMSE=11.3727, Time=9.6s

🏆 BEST ADVANCED MODEL: XGBoost_Tuned
🎯 Best RMSE: 10.8094
📈 Final training on advanced features...
